## Packages Needed

In [1]:
!pip install --upgrade plotly
!pip install pyspark
!pip install kaggle

### Imports 

In [3]:
import os
import pandas as pd
import pyspark
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Setting the correct working directory and putting the kaggle.json file in 
## '/root/.kaggle/' directory or we wont be able to access Kaggle API

In [12]:
# os.chdir('/.kaggle')


In [7]:
os.chdir('/root/.kaggle/')

!cp '/content/kaggle.json' '/root/.kaggle/'
# !ls

# count = 0
# for root, folders, filenames in os.walk('/root/'):
#    print(root, folders)


In [8]:
!chmod 600 kaggle.json

## Loading the Tokyo Olympics dataset using Kaggle API

In [9]:
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo
!unzip 2021-olympics-in-tokyo

  0% 0.00/348k [00:00<?, ?B/s]
100% 348k/348k [00:00<00:00, 51.4MB/s]
Archive:  2021-olympics-in-tokyo.zip
  inflating: Athletes.xlsx           
  inflating: Coaches.xlsx            
  inflating: EntriesGender.xlsx      
  inflating: Medals.xlsx             
  inflating: Teams.xlsx              


In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 61 kB/s 
     |████████████████████████████████| 198 kB 47.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=21c5d0df5540ce73bc149ac922103b0cc8203be7fa4e1025ae35cdee00ac0c3f
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


## Creating a Spark Session

In [10]:
# spark.stop()
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

## Converting the excel files to csv files

In [11]:
def xlsx_to_csv(filename):
  pd.read_excel(filename+'.xlsx').to_csv(filename+'.csv', sep = ',', index=False)

xlsx_to_csv('Athletes')
xlsx_to_csv('Coaches')
xlsx_to_csv('EntriesGender')
xlsx_to_csv('Medals')
xlsx_to_csv('Teams')

In [12]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entriesgender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')

In [85]:
entriesgender = pd.read_csv('EntriesGender.csv')

In [13]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [14]:
coaches.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [70]:
entriesgender.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [16]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


## Sorting the Medals table based on the total number of medals

In [17]:
medals.sort_values('Rank by Total', inplace=True)
medals.head(10)

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5
5,6,Australia,17,7,22,46,6
9,10,Italy,10,10,20,40,7
8,9,Germany,10,11,16,37,8
6,7,Netherlands,10,12,14,36,9
7,8,France,10,12,11,33,10


## Visualizing the number of Gold, Silver, and Bronze medals each country won

In [18]:
fig = px.bar(medals, x='Team/NOC', y=['Gold', 'Silver', 'Bronze'], color_discrete_sequence=['Gold', 'Silver', 'Brown'], title = 'Medals Earned By Respective Countrys')
fig.show()

In [19]:
teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [20]:
spark.read.csv('Coaches.csv', header=True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('select * from Coaches').show(10)

+--------------------+--------------------+-----------------+--------+
|                Name|                 NOC|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       Côte d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
+--------------------+--------------------+-----------------+--------+
only s

In [21]:
pd_Coaches_query = spark.sql('select NOC, COUNT(NOC) as Count from Coaches group by NOC order by NOC')
pd_Coaches_query.show(5)
pd = pd_Coaches_query.toPandas()

+---------+-----+
|      NOC|Count|
+---------+-----+
|   Angola|    1|
|Argentina|   11|
|Australia|   22|
|  Austria|    3|
|  Bahrain|    1|
+---------+-----+
only showing top 5 rows



In [22]:
pd.head()

,NOC,Count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1


In [23]:
fig = px.bar(pd, x='NOC', y='Count', color = 'Count', title='Count of Coaches from each Country')
fig.show()

In [24]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [25]:
spark.read.csv('Athletes.csv', header=True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('select * from Athletes order by NOC, Discipline').show(10)

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
+--------------------+-----------+-------------------+
only showing top 10 rows



In [26]:
pd_athletes = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
             count(*) as Count from athletes group by NOC, Discipline
             order by NOC, Discipline''').toPandas()

In [27]:
pd_athletes.head(30)

,NOC,Discipline,Count
0,Afghanistan,Athletics,2
1,Afghanistan,Shooting,1
2,Afghanistan,Swimming,1
3,Afghanistan,Taekwondo,1
4,Albania,Artistic Gymnastics,1
5,Albania,Athletics,1
6,Albania,Judo,1
7,Albania,Shooting,1
8,Albania,Swimming,2
9,Albania,Weightlifting,2


In [28]:
fig = px.bar(pd_athletes, x='NOC', y='Count', color='Discipline', title='Number of Athletes from each country in each sport categor')
fig.show()

## In the above figure, as there are many sport categories we are not able to see anything properly. So I am going to check the Disciplines with large number of participants

In [29]:
athletes['Discipline'].value_counts()

Athletics                2068
Swimming                  743
Football                  567
Rowing                    496
Hockey                    406
Judo                      373
Handball                  343
Shooting                  342
Sailing                   336
Rugby Sevens              283
Basketball                280
Wrestling                 279
Volleyball                274
Boxing                    270
Water Polo                269
Fencing                   249
Equestrian                237
Canoe Sprint              236
Baseball/Softball         220
Cycling Track             208
Cycling Road              190
Weightlifting             187
Artistic Gymnastics       187
Tennis                    178
Badminton                 164
Table Tennis              164
Diving                    133
Taekwondo                 123
Archery                   122
Golf                      115
Triathlon                 106
Artistic Swimming          98
Rhythmic Gymnastics        95
Beach Voll

In [30]:
athletes['Discipline'].value_counts().head(5)

Athletics    2068
Swimming      743
Football      567
Rowing        496
Hockey        406
Name: Discipline, dtype: int64

In [32]:
pd_athletes = pd_athletes.loc[pd_athletes['Discipline'].isin(['Athletics', 'Swimming', 'Football'])]
pd_athletes

,NOC,Discipline,Count
0,Afghanistan,Athletics,2
2,Afghanistan,Swimming,1
5,Albania,Athletics,1
8,Albania,Swimming,2
10,Algeria,Athletics,8
...,...,...,...
2103,Zambia,Athletics,2
2105,Zambia,Football,21
2107,Zambia,Swimming,2
2108,Zimbabwe,Athletics,1


In [33]:
pd_athletes.Discipline.value_counts()

Athletics    195
Swimming     181
Football      24
Name: Discipline, dtype: int64

In [34]:
fig = px.bar(pd_athletes, x='NOC', y='Count', color='Discipline', title='Number of Athletes from each country in each sport categor')
fig.show()

In [72]:
entriesgender.sort_values('Total', ascending=0)

,Discipline,Female,Male,Total
4,Athletics,969,1072,2041
36,Swimming,361,418,779
20,Football,264,344,608
29,Rowing,257,265,522
24,Judo,192,201,393
23,Hockey,192,192,384
32,Shooting,178,178,356
31,Sailing,175,175,350
22,Handball,168,168,336
30,Rugby Sevens,146,151,297


In [83]:
entriesgender.sort_values('Total', ascending=False, inplace=True)

In [86]:
specs = [[{'type':'domain'}, {'type':'domain'}]] *10
fig = make_subplots(rows=10, cols=2, subplot_titles=entriesgender['Discipline'], specs=specs)
sub_figs = []
for i, row in entriesgender.iterrows():
  sub_figure = go.Pie(labels=['Female', 'Male'], values = [row['Female'], row['Male']])
  sub_figs.append(sub_figure)


k = 0
for i in range(1, 10):
  for j in range(1, 3):
    fig.add_trace(sub_figs[k], i, j)
    k+=1
  
fig.update_layout(showlegend=False, height=5000, width = 1000, title_text="Female to Male ratio in the top 10 disciplines in Tokyo Olympics 2020")
fig.update_traces(textposition='inside', textinfo='label+percent', hoverinfo='label+value+percent')

fig = go.Figure(fig)
fig.show()